In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Simulation of scoring systems for Volleyball

This lab simulates volleyball matches between two teams. (For more information about the game of volleyball, see e.g. the [Wikipedia page](https://en.wikipedia.org/wiki/Volleyball)).

## The match

A match consists of a number of sets, best of five sets wins, with no need to play further sets if a team has won three sets.

Each set consists of a number of "rallies" (a serve with the ensuing play), the system for awarding the points differ between the "old" scoring (used until 1999), and the "new" scoring (since 1999).

## "New" scoring
1. After each rally a point is awarded to the team who wins the rally.
2. The set is played until one team reaches 25 points (and has a two point margin).
3. The next rally is served by the team that won the rally.

## "Old scoring
1. After a rally, a point is awarded to the serving team if it wins the rally.
2. If the serving team loses the rally, no point is awarded. 
3. The set is won by the first team to get 15 points.
4. The next rally is served by the team that won the rally.


Import random module, and define a helper function for the standard deviation (as usual).

In [2]:
import random

def stdDev(X):
    """Assumes X is a list of numbers,
       Returns the standard deviation of X"""
    diffSum = 0.0
    mean = sum(X)/len(X)
    for x in X:
        diffSum += (x - mean)**2
    return diffSum ** 0.5


## The class `VolleyballMatch`

Here we write an implementation of a class that simulates a single match. Each team is awarded a probability of winning a rally when that team is serving (this is of course dependent which the opposing team is).

We are making the not completely realistic assumption that each rally is an independent event.



In [3]:
class VolleyballMatch:
    def __init__(self, scoringClass, teamAProb, teamBProb): #2 scoring classes Old/New
        """scoringClass is a subclass of Set,
           teamAProb, teamBProb are the probabilities of respectively winning
           a rally when it is serving"""
        self.scoringClass = scoringClass   # either OldSet or NewSet
        self.probs = (teamAProb, teamBProb)
        self.sets = []                     # a list of the sets of the match
        self.server = random.choice([0,1]) # who is serving
        self.score = [0,0]                 # the score (in sets)

    def play(self):
        """Simulate one match"""
        while max(self.score) < 3:
            st = self.scoringClass(self.probs[0], self.probs[1], self.server)
            w = st.play()
            self.score[w] += 1  # winner gets score increases
            self.sets.append(st)
            self.server = w     # winner of last set serves in the next
        if self.score[0] == 3:
            return 0            # first team wins
        else:
            return 1            # second team wins

    def __str__(self):
        """Print out the score of the match (after it is played)"""
        sets = "{}-{}".format(self.score[0], self.score[1])
        points = ""
        for st in self.sets:
            points = points + "{}-{},".format(st.score[0], st.score[1])
        return sets + " (" + points[:-1] + ")"


## The class `Set`

Next we simulate the playing of a single set in a match. We have a common superclass `Set` with subclasses `OldSet` for the old scoring method, and `NewSet` for the new scoring method.



In [4]:
class Set:
    def __init__(self, teamAProb, teamBProb, server):
        self.probs = (teamAProb, teamBProb)
        self.score = [0,0]
        self.points = [] # A list of who is the winner of a point
        self.server = server


class NewSet(Set):
    def play(self):
        """Play a set with new scoring"""
        while True:
            if random.random() < self.probs[self.server]:#random.random() generates rand no. between 0.0 and 1.0
                # the server wins
                self.points.append(self.server)
                self.score[self.server] += 1
                self.points.append(self.server)
            else:
                # the server loses
                self.points.append(1-self.server)
                self.score[1-self.server] += 1
                self.server = 1 - self.server
                self.points.append(1-self.server)
            # has anyone won?
            if max(self.score) >= 25 and abs(self.score[0] - self.score[1]) >=2:
                # one team has reached at least 25 and leads with two points.
                break
        if self.score[0] > self.score[1]:
            return 0
        else:
            return 1


class OldSet(Set):
    def play(self):
        """Play a set with the old scoring"""
        while True:
            if random.random() < self.probs[self.server]:
                #server wins
                self.points.append(self.server)
                self.score[self.server] += 1
                self.points.append(self.server)
            
            else:
                # server loses
                self.server = 1 - self.server
            
            if max(self.score) >= 15:
                break
                
        if self.score[0] > self.score[1]:
            return 0
        else:
            return 1
            

In [5]:
match = VolleyballMatch(OldSet,0.5,0.5)
match.play()
print(match)




1-3 (10-15,15-2,11-15,13-15)


## Simulate a single match

Now, we should be able to simulate a single match. (Note that in constrast to many other sports, the serving team is at a [disadvantage in Volleyball](https://fivethirtyeight.com/features/serving-is-a-disadvantage-in-some-olympic-sports/).


In [6]:
def simMatch(scClass, probA, probB):
    """Simulate a single match with the given probabilities of winning the serve"""
    match = VolleyballMatch(NewSet, probA, probB)
    match.play()
    print(match)

simMatch(NewSet, 0.36, 0.32)

2-3 (21-25,25-19,28-26,29-31,17-25)


## Simulate a series of matches

Next we simulate a number of trials each consisting of a number of matches, both under the old and the new scoring systems.

In [7]:
def simAll(scoringClasses, probA, probB, numMatches, numTrials):
    """Simulates numTrials runs of numMatches matches with given probabilities and scoring"""
    for scClass in scoringClasses:
        teamAWins = [] # The list of frequencies of matches won by team A in the different trials
        for i in range(numTrials):
            # play numMatches matches.
            aWins = 0
            # Simulate numMatches matches
            match = VolleyballMatch(scClass, probA, probB)
            for i in range(numMatches):
                if match.play() == 0:
                    aWins += 1
           # print(aWins)
            teamAWins.append(aWins/numMatches)
        #print("{}: Team A winning frequency:  mean: {:.4f}, std. dev.: {:.4f}".format(scClass.__name__, sum(teamAWins)/numTrials, stdDev(teamAWins)))
        if len(scoringClasses)>1:
            return sum(teamAWins)/numTrials, simAll([scoringClasses[-1]], probA, probB, numMatches, numTrials)
        else:
            return sum(teamAWins)/numTrials
        


In [8]:
def OldvsNew(probA,probB):
    list = []
    Oldteam = 0
    Newteam = 0
    for i in range(25):
        for i in simAll([OldSet, NewSet], probA,probB ,2000, 100):
            list.append(i)
            if len(list)<=1:
                pass
            else:
                if list[0]>list[-1]:
                    Oldteam += 1
                else:
                    Newteam +=1
                list = []
    return(Oldteam,Newteam)


for i in range(10):
    print(OldvsNew(0.5,0.5))   

    



(13, 12)
(12, 13)
(14, 11)
(9, 16)
(9, 16)
(13, 12)
(12, 13)
(8, 17)
(9, 16)
(12, 13)


In [10]:
simAll([OldSet, NewSet], 0.5,0.5 ,2000, 100)

(0.52, 0.55)